In [143]:
import geemap
import ipywidgets as widgets
import numpy as np
import pandas as pd
import proplot as plot 
import matplotlib.pyplot as plt 
from ipygee import chart
from pandas.plotting import register_matplotlib_converters
import ee

In [144]:
style = {'description_width': 'initial'}

year = widgets.IntSlider(description='Select Year:', value=2013, min=2013, max=2021, style=style)

In [145]:
add_layer = widgets.Button(
    description='Add Layer',
    button_style='primary',
    tooltip='Click to add layer',
    style=style
)

output1 = widgets.Output()

hbox = widgets.HBox([year, add_layer])

In [146]:
Map = geemap.Map(center=[23.5121, 80.3288], zoom=5)
def submit_check(b):
  with output1:
    start_date = str(year.value)+ '-' + '01' + '-01'
    end_date = str(year.value)+ '-' + '12' + '-31'
    compare_start_date=''
    compare_end_date=''
    if year.value>2019:
      compare_start_date+='2019-01-01'
      compare_end_date+='2019-12-31'
    else:
      compare_start_date+=start_date
      compare_end_date+=end_date
    
    admin = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1");
    maharashtra = admin.filter(ee.Filter.eq('ADM1_NAME', 'Maharashtra'))
    geometry = maharashtra.geometry()

    point = ee.Geometry.Point([80.3288, 23.5121])

    image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterDate(start_date, end_date) \
        .sort('CLOUD_COVER') \
        .min() \
        .select('B[1-7]') \
        .clip(geometry)


    vis_params = {
        'min': 0,
        'max': 3000,
        'bands': ['B5', 'B4', 'B3']
    }

    Map.centerObject(point, 5)
    Map.addLayer(image, vis_params, "Landsat-8")
    modis = ee.ImageCollection('MODIS/006/MCD12Q1') \
                .filterDate('2016-01-01', '2019-01-01') \
                .select('LC_Type1') \
                .reduce(ee.Reducer.median())

    points = modis.sample(**{
        'region': ee.Geometry.Rectangle([72.0000, 23.0000, 82.0000, 15.0000]),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })

    Map.addLayer(points, {}, 'training', False)

    # Use these bands for prediction.
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


    # This property of the table stores the land cover labels.
    label = 'LC_Type1_median'
    input = image.select(bands)

    # Overlay the points on the imagery to get training.
    training = input.sampleRegions(**{
      'collection': points,
      'properties': [label],
      'scale': 30
    })

    # Train a CART classifier with default parameters.
    trained = ee.Classifier.smileCart().train(training, label, bands)

    result = image.select(bands).classify(trained)

    landcover = ee.ImageCollection('MODIS/006/MCD12Q1') \
                .filterDate(compare_start_date, compare_end_date) \
                .select('LC_Type1') \
                .reduce(ee.Reducer.mean()) \
                .clip(geometry)

    LandCoverVis = {
      'min': 1.0,
      'max': 17.0,
      'palette': [
        '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
        'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
        '69fff8', 'f9ffa4', '1c0dff'
      ],
    }

    Map.centerObject(point, 5)
    Map.addLayer(landcover, LandCoverVis, 'MODIS Land Cover')

    landsatcover = result.set('classification_class_values', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
    landsatcover = landsatcover.set('classification_class_palette', ['1c0dff', '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4',   ])
    Map.addLayer(landsatcover, {}, 'Land cover')

    ee_class_table = """

    Value	Color	Description
    0	1c0dff	Water
    1	05450a	Evergreen needleleaf forest
    2	086a10	Evergreen broadleaf forest
    3	54a708	Deciduous needleleaf forest
    4	78d203	Deciduous broadleaf forest
    5	009900	Mixed forest
    6	c6b044	Closed shrublands
    7	dcd159	Open shrublands
    8	dade48	Woody savannas
    9	fbff13	Savannas
    10	b6ff05	Grasslands
    11	27ff87	Permanent wetlands
    12	c24f44	Croplands
    13	a5a5a5	Urban and built-up
    14	ff6d4c	Cropland/natural vegetation mosaic
    15	69fff8	Snow and ice
    16	f9ffa4	Barren or sparsely vegetated
    254	ffffff	Unclassified

    """

    legend_dict = geemap.legend_from_ee(ee_class_table)
    Map.add_legend(legend_title="Land Cover Legend", legend_dict=legend_dict, position='bottomleft')
    print("Map Generated")

add_layer.on_click(submit_check)

In [147]:
output1

Output()

In [148]:
hbox

In [149]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [150]:
# Map = geemap.Map()

# admin = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1");
# maharashtra = admin.filter(ee.Filter.eq('ADM1_NAME', 'Maharashtra'))
# geometry = maharashtra.geometry()

# point = ee.Geometry.Point([80.3288, 23.5121])

# image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#     .filterDate('2019-01-01', '2019-12-31') \
#     .sort('CLOUD_COVER') \
#     .min() \
#     .select('B[1-7]') \
#     .clip(geometry)


# vis_params = {
#     'min': 0,
#     'max': 3000,
#     'bands': ['B5', 'B4', 'B3']
# }

# Map.centerObject(point, 5)
# Map.addLayer(image, vis_params, "Landsat-8")

In [151]:
# modis = ee.ImageCollection('MODIS/006/MCD12Q1') \
#               .filterDate('2016-01-01', '2019-01-01') \
#               .select('LC_Type1') \
#               .reduce(ee.Reducer.median())

# points = modis.sample(**{
#     'region': ee.Geometry.Rectangle([72.0000, 23.0000, 82.0000, 15.0000]),
#     'scale': 30,
#     'numPixels': 5000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })

# Map.addLayer(points, {}, 'training', False)


In [152]:
# # Use these bands for prediction.
# bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# # This property of the table stores the land cover labels.
# label = 'LC_Type1_median'
# input = image.select(bands)

# # Overlay the points on the imagery to get training.
# training = input.sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# # Train a CART classifier with default parameters.
# trained = ee.Classifier.smileCart().train(training, label, bands)


In [153]:
# result = image.select(bands).classify(trained)

In [154]:
# landcover = ee.ImageCollection('MODIS/006/MCD12Q1') \
#               .filterDate('2019-01-01', '2019-12-31') \
#               .select('LC_Type1') \
#               .reduce(ee.Reducer.mean()) \
#               .clip(geometry)

# LandCoverVis = {
#   'min': 1.0,
#   'max': 17.0,
#   'palette': [
#     '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
#     'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
#     '69fff8', 'f9ffa4', '1c0dff'
#   ],
# }

# Map.centerObject(point, 5)
# Map.addLayer(landcover, LandCoverVis, 'MODIS Land Cover')

In [155]:
# landsatcover = result.set('classification_class_values', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
# landsatcover = landsatcover.set('classification_class_palette', ['1c0dff', '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4',   ])
# Map.addLayer(landsatcover, {}, 'Land cover')

In [156]:
# ee_class_table = """

# Value	Color	Description
# 0	1c0dff	Water
# 1	05450a	Evergreen needleleaf forest
# 2	086a10	Evergreen broadleaf forest
# 3	54a708	Deciduous needleleaf forest
# 4	78d203	Deciduous broadleaf forest
# 5	009900	Mixed forest
# 6	c6b044	Closed shrublands
# 7	dcd159	Open shrublands
# 8	dade48	Woody savannas
# 9	fbff13	Savannas
# 10	b6ff05	Grasslands
# 11	27ff87	Permanent wetlands
# 12	c24f44	Croplands
# 13	a5a5a5	Urban and built-up
# 14	ff6d4c	Cropland/natural vegetation mosaic
# 15	69fff8	Snow and ice
# 16	f9ffa4	Barren or sparsely vegetated
# 254	ffffff	Unclassified

# """

# legend_dict = geemap.legend_from_ee(ee_class_table)
# Map.add_legend(legend_title="Land Cover Legend", legend_dict=legend_dict, position='bottomleft')

In [157]:
# Map